Skeleton Code

In [1]:
#!pip install -qU langchain-community faiss-cpu

In [2]:
from dotenv import load_dotenv 
import os 
load_dotenv(".env")
UPSTAGE_API_KEY = os.getenv("API_KEY")

In [3]:
# 1. Load Documents
from langchain_upstage import UpstageLayoutAnalysisLoader
layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,
                                     file_path="./data/ewha.pdf",
                                     output_type="text",
                                     split="page") # Modified
docs = layzer.load()
print(f"문서의 페이지수: {len(docs)}")

/home/yoojinoh/.miniconda3/envs/deeplearning/lib/python3.8/site-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/home/yoojinoh/.miniconda3/envs/deeplearning/lib/python3.8/site-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


문서의 페이지수: 54


In [4]:
# 2. Split documents
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter 
# 단락 -> 문장 -> 단어 순서로 재귀적으로 분할
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size = 1000, # # of tokens (한번에 몇개의 토큰들을 볼것인가)
    chunk_overlap = 100, # overlap of chunks (몇개의 token은 겹치면서 chunk를 만들지) -> 커지면 #splits증가
    language=Language.HTML 
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))
print("Type :", type(splits[0]))

Splits: 72
Type : <class 'langchain_core.documents.base.Document'>


In [5]:
splits[10]

Document(metadata={'page': 6}, page_content='이화여자대학교 학칙 2017.2.8.) ③ 복수전공을 이수하는 학생, 「고등교육법 시행령」 제29조제2항의제6호 및 제7호에 해당\n하는 학생의 재학연한은 9년을 초과할 수 없다. (개정 2014.11.21)\n④ 재입학한 학생의 재학연한은 재입학전 재학기간을 산입하여 제1항부터 제3항까지의 해\n당 기간을 초과할 수 없다. (개정 2014.11.21., 2017.8.16.)\n⑤ 제15조의 규정에 의하여 편입학한 학생의 재학연한은 잔여수업연한의 2배를 초과할 수\n없다. 다만, 이 경우 복수전공을 이수하는 학생은 잔여수업연한의 2배에 1년을 더한 연한\n을 초과할 수 없다. (개정 2014.11.21)\n⑥ 휴학기간은 재학연한에 산입하지 아니한다. (개정 2014.11.21) 제7장 교내전학 제25조(전과) ① 학생이 소속 학부 또는 학과의 변경(이하 “전과”라 한다)을 원할 때에는 「고\n등교육법 시행령」 제29조제3항의 규정이 정하는 범위내에서 허가할 수 있다. (개정\n1999.2.9., 2015.9.18.) ② 전과의 허가는 매 학년도말에 할 수 있다. (개정 1999.2.9., 2017.8.16.)\n③ 전과의 기준․대상․절차 등 세부사항은 총장이 따로 정한다.\n(개정 1992.3.16) 제8장 휴학, 복학, 제적, 자퇴 및 재입학 (개정 2017.8.16.) 제26조(휴학) ① 질병 기타 부득이한 사정으로 3주일 이상 수강할 수 없는 자는 총장의 허가\n를 얻어 휴학할 수 있다. ② 총장은 건강상의 이유로 정상적인 수업을 받을 수 없다고 인정되는 자에 대하여 휴학을\n명할 수 있다. (개정 1988.7.28)\n③ 1회의 휴학기간은 1년 이내로 한다. 다만, 교과과정상의 필요에 따라 총장이 지정하는\n학부, 학과 또는 전공에 있어서는 이를 1년으로 한다. (개정 1996.2.15)\n④ 휴학기간은 통산하여 3년(건축학전공의 경우 4년, 의예과의 경우 3학기)을 초과할

In [6]:
from langchain_upstage import UpstageEmbeddings
from langchain_community.vectorstores import FAISS 
# 3. embedding 생성
upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model = 'solar-embedding-1-large')
# 4. DB 생성 및 저장
vectorstore = FAISS.from_documents(documents=splits,
                                   embedding=upstage_embeddings) 
# 5. Retriever 생성
retriever = vectorstore.as_retriever() 

In [7]:
# Query 넣어서 chunk결과 확인
retriever.invoke("학기말 성적이 몇점 이하면 학사경고를 받나요?")

[Document(metadata={'page': 10}, page_content='이화여자대학교 학칙 과목 담당교수는 출석으로 인정할 수 있다. (신설 2016.6.16., 개정 2017.8.16.) 1. 중대한 질병으로 인한 경우\n2. 직계존비속의 사망으로 인한 경우\n3. 주관 기관장이 증빙서류를 발행한 국제대회, 훈련, 연수 및 교육실습 등의 참가에 의한\n경우\n4. 그 밖에 총장 또는 교과목 담당교수가 필요하다고 인정하여 허가한 경우\n③ 체육특기자의 대회출전 및 훈련으로 인한 출석대체인정은 1학기 수업시간의 2분의 1\n이내로 한다. 출석대체인정을 받는 체육특기자의 기준은 총장이 별도로 정한다. (신설\n2017.8.16.)\n[제목개정 2016.6.16.] 제41조(성적불량학생에 대한 조치) ① 학기말의 평균성적이 1.60 미만인 학생은 지도교수,\n전공주임교수, 학과장 또는 학부장의 특별지도를 받아야 한다. (개정 1996.2.15.,\n2014.9.26., 2017.8.16.) ② 삭제 (1988.7.28)\n③ 삭제 (1988.7.28)\n④ 학기말의 평균성적이 1.60 미만인 학생에게는 학사경고를 하고, 학사경고를 연속 3회\n받은 학생은 제적한다. 다만, 학사경고를 연속 3회 받아 제적된 자 또는 학사경고를 연속\n2회 받고 다른 사유로 제적 또는 자퇴하였다가 재입학한 학생은 재입학한 후부터 학사경고\n를 1회 받은 때에는 제적한다. (개정 2002.11.8., 2017.8.16.)\n⑤ 성적이 불량하여 한 학년의 전 교과목 또는 일부 교과목을 재수강하여야 할 때에는 학\n년단위로 2회에 한하여 유급시킬 수 있으며, 재학기간중 2회를 초과할 때에는 제적한다.\n유급에 관한 사항은 따로 정한다. (개정 1988.7.28., 2015.9.18.)\n⑥ 제24조의 규정에 의한 재학연한의 만료시까지 총평균성적이 1.70미만인 학생에 대하여\n는 졸업을 인정하지 아니한다. (개정 2014.11.21) 제41조의2 삭제 (1988.7.28)\n제

In [9]:
# 6. Prompt 생성
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Answer in Korean.
    ---
    #Question:{question}
    ---
    #Context:{context}
    ---
    #Answer:
    """
)


In [10]:
# 7. LLM 생성
from langchain_upstage.chat_models import ChatUpstage 
llm = ChatUpstage(api_key=UPSTAGE_API_KEY, temperature=0) 

In [13]:
# 8. chain 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm 
    | StrOutputParser() 
)

In [14]:
# Run Chain ! 만든 Chain 바탕으로 쿼리 입력하고 실행
question = "학기말 성적이 몇점 이하면 학사경고를 받나요?"
response = chain.invoke(question) # RunnablePassthrough라서
response

'학기말 성적이 1.60점 미만이면 학사경고를 받습니다.'